# Progetto Few-Shot Logo Recognition: Valutazione Baseline ResNet50

### 1. Abstract e Obiettivo
Questo notebook ha l'obiettivo di stabilire una **baseline (linea di base)** per il task di *Few-Shot Logo Recognition* utilizzando il dataset **LogoDet-3K**.
Prima di procedere con il fine-tuning o la modifica dell'architettura della rete, è fondamentale misurare le performance di un modello standard pre-addestrato. I risultati ottenuti in questo notebook serviranno come metro di paragone per valutare l'efficacia delle future ottimizzazioni.

### 2. Metodologia
L'approccio utilizzato in questa fase è di tipo **"Off-the-shelf Feature Extraction"**:
* **Modello:** Viene utilizzata una **ResNet50** pre-addestrata su *ImageNet*.
* **Feature Extraction:** Il livello di classificazione finale (Fully Connected) viene rimosso e sostituito con una funzione identità. Invece di predire le 1000 classi di ImageNet, il modello restituisce il vettore delle feature (embedding) di dimensione **2048**.
* **Inferenza:** La classificazione avviene tramite **Similarità del Coseno**. Viene calcolata la distanza tra l'embedding dell'immagine di *query* (da classificare) e l'embedding dell'immagine di *supporto* (esempio noto del brand).
* **Protocollo:** Il testing segue un approccio *episodico* (N-Way, K-Shot) simulato su 1000 episodi per garantire robustezza statistica.

### 3. Struttura del Notebook
Il codice è organizzato nelle seguenti sezioni logiche:

1.  **Configurazione e Setup:** Definizione dei parametri globali (seed, path, device) e setup dell'ambiente (Google Drive).
2.  **Gestione del Dataset:**
    * `DatasetTest`: Classe PyTorch per il caricamento delle immagini.
    * `FewShotIterator`: Logica per la creazione degli episodi (Support Set vs Query Set).
3.  **Metriche di Valutazione:** Implementazione della classe `MetricEvaluator` per il calcolo di Accuracy, mAP, F1-Score, Precision, Recall e Discriminant Ratio (J).
4.  **Definizione Modello Baseline:** Funzione `get_baseline_resnet50` per il caricamento del modello e la modifica dell'head.
5.  **Loop di Valutazione:** Esecuzione del test e aggregazione dei risultati finali.

---
**Dataset:** LogoDet-3K | **Modello:** ResNet50 (Frozen) | **Metrica Principale:** Cosine Similarity

In [1]:
import torch
import sys
import os
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import glob
import random
import xml.etree.ElementTree as ET
from itertools import cycle
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.models as models
from torchvision.models import ResNet50_Weights
from google.colab import drive
import zipfile
import pandas as pd

class Config:
    # 1. SETUP
    project_name = "FewShot_Evaluation"
    seed = 42
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # 2. DATASET PATH
    dataset_root = "LogoDet-3K/LogoDet-3K-divided"
    csv_index_path = "LogoDet-3K/brand_to_index.csv"

    # 3. MODEL PARAMETERS
    embedding_dim = 128
    pretrained = True
    freeze_layers = 5
    trained_model_path = ""

    # 4. EVALUATION SETTINGS
    prediciton_threashold = 0.5
    n_shot = 1
    num_episodes = 1000

torch.manual_seed(Config.seed)
random.seed(Config.seed)

def setup_dataset(zip_path, extract_to):
    """
    Mounts Google Drive and extracts the dataset if not already present.
    """
    # 1. Mount Google Drive
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')

    # 2. Check if the folder already exists
    if os.path.exists(extract_to):
        print(f"Dataset folder '{extract_to}' already exists. Skipping extraction.")
    else:
        print(f"Extracting dataset from {zip_path}...")
        if os.path.exists(zip_path):
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(extract_to)
            print("Extraction complete.")
        else:
            print(f"ERROR: Zip file not found at {zip_path}. Check your path.")

setup_dataset("/content/drive/MyDrive/LogoDet-3K-divided.zip", "/content/LogoDet-3K")

ModuleNotFoundError: No module named 'torch'

In [ ]:
class DatasetTest(Dataset):
    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform

    # Load label string to index mapping
        df = pd.read_csv(Config.csv_index_path)
        self.label_to_id = {row['brand']: int(row['index']) for _, row in df.iterrows()}


    def __len__(self):
        return len(self.file_list)

    def load_image(self, image_path):
        img = Image.open(image_path)
        if self.transform:
            img = self.transform(img)
        image_label = os.path.basename(os.path.dirname(image_path))

        label_idx = self.label_to_id[image_label]

        return {"image": img, "label": label_idx}

    def __getitem__(self, idx):
        return self.load_image(self.file_list[idx])


def getTestPaths(root_dir, total_set_size=None, min_images_per_brand=2):
    test_path = os.path.join(root_dir, 'test')
    test_brand_list = []

    # Collect brand folders
    if not os.path.exists(test_path):
        print(f"Warning: {test_path} not found.")
        return []

    for category in os.listdir(test_path):
        cat_path = os.path.join(test_path, category)
        if os.path.isdir(cat_path):
            for brand in os.listdir(cat_path):
                brand_full_path = os.path.join(cat_path, brand)
                if os.path.isdir(brand_full_path):
                    test_brand_list.append(brand_full_path)

    test_data_list = []

    # Sampling Logic
    if total_set_size is not None:
        images_per_brand = round(total_set_size / len(test_brand_list))

        if images_per_brand < min_images_per_brand:
            new_test_brand_count = round(total_set_size / min_images_per_brand)
            test_brand_list = random.sample(test_brand_list, min(len(test_brand_list), new_test_brand_count))
            images_per_brand = min_images_per_brand

        for brand in test_brand_list:
            imgs = glob.glob(os.path.join(brand, '*.jpg'))

            if len(imgs) < min_images_per_brand:
                print(f"images are less than {min_images_per_brand} for this brand: {brand} in the TEST set")

            test_data_list.extend(random.sample(imgs, min(images_per_brand, len(imgs))))
    else:
        for brand in test_brand_list:
            test_data_list.extend(glob.glob(os.path.join(brand, '*.jpg')))

    return test_data_list

: 

In [ ]:
def compute_global_embeddings(model, file_list, transform, device):
    model.eval()
    all_embeddings = []
    all_labels = []

    # Simple linear dataset of all unique test images
    dataset = DatasetTest(file_list, transform)
    loader = DataLoader(dataset, batch_size=64, shuffle=False)

    with torch.no_grad():
        for data in loader:
            images = data["image"].to(device)
            labels = data["label"]

            # Extract and move to CPU to save VRAM
            embeddings = F.normalize(model(images), p=2, dim=1).cpu()

            all_embeddings.append(embeddings)
            all_labels.append(labels)

    return torch.cat(all_embeddings), torch.cat(all_labels)

def cosine_similarity(averaged_support_embeddings, query_embeddings_tensor):

    # Normalize embeddings if you want cosine similarity
    support_emb_norm = F.normalize(averaged_support_embeddings, p=2, dim=0)       # [embedding_dim]
    query_emb_norm = F.normalize(query_embeddings_tensor, p=2, dim=1)             # [num_queries, embedding_dim]

    # Compute cosine similarity
    sims = torch.matmul(query_emb_norm, support_emb_norm)  # [num_queries]
    return sims

def evaluate_few_shot(model, fewshot_iterator, transform, device, num_episodes=100):
    evaluator = MetricEvaluator(device=device)
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    r_at_95p = []
    ap_scores = []

    torch.cuda.empty_cache()

    unique_paths = list(fewshot_iterator.all_files_set)
    embs, labels = compute_global_embeddings(model, unique_paths, transform, device)
    j_score = evaluator.compute_discriminant_ratio(embs, labels)

    # 2. Set to eval mode and disable gradient tracking
    model.eval()
    with torch.no_grad():
        for i in range(num_episodes):

            task = fewshot_iterator()

            if task is None:
                print(f"Stopped early at episode {i} because we ran out of brands.")
                break

            support_paths = task["support_set"]
            query_paths = task["query_set"]

            # Build datasets and loaders
            support_dataset = DatasetTest(support_paths, transform)
            query_dataset = DatasetTest(query_paths, transform)

            support_loader = DataLoader(support_dataset, batch_size=32)
            query_loader = DataLoader(query_dataset, batch_size=64)

            # Extract embeddings
            support_embeddings = []
            query_embeddings = []
            query_labels = []
            support_labels = []

            # Compute embeddings for support set
            for data in support_loader:
                images = data["image"].to(device)
                support_embeddings.append(F.normalize(model(images), p=2, dim=1))

                batch_labels = data["label"]
                support_labels.append(batch_labels)
                support_brand = batch_labels[0]

            support_embeddings_tensor = torch.cat(support_embeddings)
            support_labels_tensor = torch.cat(support_labels)

            # Average embeddings
            averaged_support_embeddings = support_embeddings_tensor.mean(dim=0)
            averaged_support_embeddings_for_ap = support_embeddings_tensor.mean(dim=0, keepdim=True)

            # Compute embeddings for query set
            for data in query_loader:
                images = data["image"].to(device)
                query_embeddings.append(F.normalize(model(images), p=2, dim=1))

                batch_labels = data["label"]
                query_labels.append(batch_labels)

            # query_embeddings and query_labels are list of tensors, this unrolls them
            query_embeddings_tensor = torch.cat(query_embeddings)
            query_labels_tensor = torch.cat(query_labels)

            # mAP
            ap_score_single = evaluator.compute_map(
                query_emb=averaged_support_embeddings_for_ap,
                gallery_emb=query_embeddings_tensor,
                query_labels=support_labels_tensor,
                gallery_labels=query_labels_tensor
            )

            ap_scores.append(ap_score_single)

            # Compute similarity
            sims = cosine_similarity(averaged_support_embeddings, query_embeddings_tensor)

            # Ground truth: query belongs to support brand?
            gt = (query_labels_tensor == support_brand).float()

            # Predictions, does the model predict it is the same brand?
            pred = (sims >= Config.prediciton_threashold).float().cpu()

            # Accuracy
            acc = (pred == gt).float().mean().item()
            accuracies.append(acc)

            # Precision, Recall, F1
            prec, rec = evaluator.compute_precision_recall(sims, gt, threshold=Config.prediciton_threashold)
            f1 = evaluator.compute_f1_score(prec, rec)
            r95 = evaluator.compute_recall_at_fixed_precision(sims, gt, min_precision=0.95)

            precisions.append(prec)
            recalls.append(rec)
            f1_scores.append(f1)
            r_at_95p.append(r95)

    # Aggregate results
    results = {
        "accuracy": sum(accuracies) / len(accuracies),
        "precision": sum(precisions) / len(precisions),
        "recall": sum(recalls) / len(recalls),
        "f1": sum(f1_scores) / len(f1_scores),
        "r@95p": sum(r_at_95p) / len(r_at_95p),
        "map": sum(ap_scores) / len(ap_scores),
        "J": j_score,
    }
    return results

class MetricEvaluator:
    """
    A class to calculate evaluation metrics for Few-Shot Learning and Metric Learning.

    Implements:
    1. Discriminant Ratio (J): Optimized scalar implementation (O(d) memory).
    2. Mean Average Precision (mAP): Ranking quality metric.
    3. Recall at Fixed Precision (R@P): Operational metric.
    4. Precision & Recall: Raw metrics at a specific similarity threshold.
    5. F1 Score: Harmonic mean of Precision and Recall.
    """

    def __init__(self, device=None):
        """
        Initialize the evaluator.

        Args:
            device (str): 'cuda' or 'cpu'. If None, detects automatically.
        """
        if device:
            self.device = device
        else:
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

        self.epsilon = 1e-6  # For numerical stability

    def compute_discriminant_ratio(self, embeddings, labels):
        """
        Calculates the Discriminant Ratio (J) using the optimized Scalar approach.

        Theory:
            J = Tr(Sb) / Tr(Sw)
            Using the Trace Trick: Tr(Sw) = Tr(St) - Tr(Sb)

        Args:
            embeddings (torch.Tensor): Tensor of shape (Batch_Size, Dimension).
            labels (torch.Tensor): Tensor of class labels.

        Returns:
            float: The Discriminant Ratio score.
        """
        embeddings = embeddings.to(self.device)
        labels = labels.to(self.device)

        # 1. Global Mean Computation
        global_mean = embeddings.mean(dim=0)

        # 2. Calculate Trace of Total Scatter (St)
        # Sum of squared Euclidean distances of all points from the global mean.
        tr_st = torch.sum((embeddings - global_mean) ** 2)

        # 3. Calculate Trace of Between-Class Scatter (Sb)
        tr_sb = 0
        unique_classes = torch.unique(labels)

        for c in unique_classes:
            class_mask = (labels == c)
            class_embeddings = embeddings[class_mask]
            n_c = class_embeddings.size(0)

            if n_c > 0:
                mu_c = class_embeddings.mean(dim=0)
                tr_sb += n_c * torch.sum((mu_c - global_mean) ** 2)

        # 4. Calculate Trace of Within-Class Scatter (Sw)
        tr_sw = tr_st - tr_sb

        # Calculate J
        j_score = tr_sb / (tr_sw + self.epsilon)

        return j_score.item()

    def compute_map(self, query_emb, gallery_emb, query_labels, gallery_labels):
        """
        Calculates Mean Average Precision (mAP).
        """
        query_emb = query_emb.to(self.device)
        gallery_emb = gallery_emb.to(self.device)
        query_labels = query_labels.to(self.device)
        gallery_labels = gallery_labels.to(self.device)

        # L2 Normalize for Cosine Similarity
        query_emb = F.normalize(query_emb, p=2, dim=1)
        gallery_emb = F.normalize(gallery_emb, p=2, dim=1)

        # Similarity Matrix: S = Q * G^T
        similarity_matrix = torch.matmul(query_emb, gallery_emb.T)

        num_queries = query_labels.size(0)
        average_precisions = []

        for i in range(num_queries):
            scores = similarity_matrix[i]
            target_label = query_labels[i]

            # Ranking
            sorted_indices = torch.argsort(scores, descending=True)
            sorted_gallery_labels = gallery_labels[sorted_indices]

            # Relevance Mask
            relevance_mask = (sorted_gallery_labels == target_label).float()

            total_relevant = relevance_mask.sum()
            if total_relevant == 0:
                average_precisions.append(0.0)
                continue

            # Cumulative Precision
            cumsum = torch.cumsum(relevance_mask, dim=0)
            ranks = torch.arange(1, len(relevance_mask) + 1).to(self.device)
            precisions = cumsum / ranks

            # Average Precision (AP)
            ap = (precisions * relevance_mask).sum() / total_relevant
            average_precisions.append(ap.item())

        if not average_precisions:
            return 0.0
        return sum(average_precisions) / len(average_precisions)

    def compute_precision_recall(self, similarity_scores, is_match, threshold=0.5):
        """
        Calculates raw Precision and Recall at a specific similarity threshold.

        Definitions:
            Precision = TP / (TP + FP)
            Recall    = TP / (TP + FN)

        Args:
            similarity_scores (torch.Tensor): 1D tensor of scores (0.0 to 1.0).
            is_match (torch.Tensor): 1D binary tensor (Ground Truth).
            threshold (float): Cutoff for deciding if a retrieval is Positive.

        Returns:
            tuple: (precision, recall)
        """
        similarity_scores = similarity_scores.to(self.device)
        is_match = is_match.to(self.device)

        # Binarize predictions: 1 if score >= threshold (Positive), else 0 (Negative)
        predicted_positive = (similarity_scores >= threshold).float()

        # True Positives (TP): Predicted Positive AND Actually Match
        tp = (predicted_positive * is_match).sum()

        # False Positives (FP): Predicted Positive BUT Actually Non-Match
        fp = (predicted_positive * (1 - is_match)).sum()

        # False Negatives (FN): Predicted Negative BUT Actually Match
        # (We invert the prediction mask to find negatives)
        fn = ((1 - predicted_positive) * is_match).sum()

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        return precision.item(), recall.item()

    def compute_recall_at_fixed_precision(self, similarity_scores, is_match, min_precision=0.95):
        """
        Calculates Recall at a Fixed Precision (R@P).
        Finds the lowest threshold where Precision >= min_precision.
        """
        similarity_scores = similarity_scores.to(self.device)
        is_match = is_match.to(self.device)

        sorted_indices = torch.argsort(similarity_scores, descending=True)
        sorted_matches = is_match[sorted_indices]

        tps = torch.cumsum(sorted_matches, dim=0)
        total_retrieved = torch.arange(1, len(sorted_matches) + 1).to(self.device)

        precisions = tps / total_retrieved

        # Find indices where Precision satisfies the constraint
        valid_indices = torch.where(precisions >= min_precision)[0]

        if len(valid_indices) == 0:
            return 0.0

        cutoff_index = valid_indices[-1]

        # Recall = TP_at_cutoff / Total_Relevant_In_Dataset
        total_relevant_in_dataset = is_match.sum()

        if total_relevant_in_dataset == 0:
            return 0.0

        recall = tps[cutoff_index] / total_relevant_in_dataset

        return recall.item()

    def compute_f1_score(self, precision, recall):
        """
        Calculates F1 Score (Harmonic Mean).
        """
        if (precision + recall) == 0:
            return 0.0
        return 2 * (precision * recall) / (precision + recall)



class FewShotIterator:
    def __init__(self, file_list, n_shot):
        """
        Initializes the iterator class.
        It prepares the global testset and creates a cyclic iterator over the valid brands.
        """
        self.n_shot = n_shot

        # 1. Validation: Check if input list is empty
        if not file_list:
            raise ValueError("The test file list is empty.")

        #    (Dataset - SupportSet)  is significantly faster with sets (O(1)) compared to lists.
        self.all_files_set = set(file_list)

        # 3. Organize data by Brand
        #    We create a dictionary mapping: { 'BrandName': [list_of_image_paths] }
        self.brands_map = {}

        for file_path in file_list:
            # Extract brand name assuming structure: .../Category/Brand/Image.jpg
            brand_name = os.path.basename(os.path.dirname(file_path))

            if brand_name not in self.brands_map:
                self.brands_map[brand_name] = []
            self.brands_map[brand_name].append(file_path)

        self.valid_brands_list = list(self.brands_map.keys())

        if not self.valid_brands_list:
            raise ValueError(f"No brand found with more than {n_shot} images.")

        #    'itertools.cycle' creates an infinite loop over the valid brands list.
        self.brand_iterator = cycle(self.valid_brands_list)

    def __call__(self):
        """
        Executed when the class instance is called.
        Logic:
        1. Pick next brand (Sequential).
        2. Pick Support Set (Random 5 images from that brand).
        3. Pick Query Set (EVERYTHING else in the testset).
        """
        # A. Get the next brand sequentially from the cycle
        try:
            selected_brand_name = next(self.brand_iterator)
        except StopIteration:
            # Gracefully signal that we are done
            print("Iterator finished: All brands have been processed.")
            return None

        # B. Retrieve all images specific to this chosen brand
        images_of_current_brand = self.brands_map[selected_brand_name]

        # Select a random number between 1 to 5 which is the number of images of the support brand guaranteed in the query set
        num_query_guarantee_if_available = random.randint(1, 5)

        # C. Create SUPPORT SET
        #    Select 'n_shot' unique images randomly from the current brand.
        support_set_list = random.sample(images_of_current_brand, self.n_shot)
        support_set_set = set(support_set_list)

        # D. Create QUERY SET (Global Subtraction)
        #    Requirement: The Query Set contains 50 images of which at least 1 is from the support brand
        #    Step 1: initialize the Query list
        query_set_list = []

        #    Step 2: Sample randomly the images to guarantee in the query set
        remaining_brand_images = list(set(images_of_current_brand) - support_set_set)
        guaranteed_images_in_query = random.sample(remaining_brand_images, min(num_query_guarantee_if_available, len(remaining_brand_images)))

        if (len(remaining_brand_images) == 0):
            print(f"for the brand {selected_brand_name} {len(remaining_brand_images)} images where put in the query set")

        #    Step 3: Sample the Negative Queries (Distractors from OTHER brands)
        # We subtract ALL images of the current brand to ensure zero accidental matches
        remaining_images_in_query = list(self.all_files_set - set(images_of_current_brand) - set(guaranteed_images_in_query))

        total_query_size = 50
        num_remaining_query = total_query_size - min(num_query_guarantee_if_available, len(remaining_brand_images))
        query_remaining = random.sample(remaining_images_in_query,  min(len(remaining_images_in_query),num_remaining_query))

        #    Step 4: Combine and Shuffle
        query_set_list = guaranteed_images_in_query + query_remaining
        random.shuffle(query_set_list)


        return {
            "brand_name": selected_brand_name,
            "support_set": support_set_list,
            "query_set": query_set_list
        }


In [ ]:



def get_baseline_resnet50(device):
    """
    Scarica la ResNet50 originale pre-addestrata su ImageNet.
    Sostituisce il layer fully connected (fc) con Identity per restituire
    i feature embeddings (dimensione 2048) invece delle classi.
    """
    print("Scaricamento ResNet50 originale (ImageNet Weights)...")
    
    # 1. Carica i pesi di default (ImageNet)
    weights = ResNet50_Weights.DEFAULT
    model = models.resnet50(weights=weights)
    
    # 2. Rimuovi la testa di classificazione (1000 classi)
    # Sostituendola con Identity, l'output del modello sarà l'output 
    # del layer precedente (avgpool), ovvero un vettore di dimensione 2048.
    model.fc = nn.Identity()
    
    # 3. Configurazione per inferenza
    model.to(device)
    model.eval() # Imposta batchnorm e dropout in modalità valutazione
    
    return model

def main():
    # 1. Configurazione Dispositivo e Trasformazioni
    # Usa le stesse trasformazioni standard di ImageNet usate nel training/testing
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    device = torch.device(Config.device if hasattr(Config, 'device') else 'cuda')

    # 2. Istanzia il modello Baseline
    model = get_baseline_resnet50(device)

    # 3. Preparazione Dati
    # Recupera i path e crea l'iteratore usando le tue funzioni esistenti
    print("Preparazione dati per il testing...")
    test_paths = getTestPaths(Config.dataset_root) 
    iterator = FewShotIterator(test_paths, n_shot=Config.n_shot)

    # 4. Esecuzione del Loop di Testing
    # Il loop 'evaluate_few_shot' riceverà vettori da 2048 dimensioni.
    # Funzionerà correttamente perché usa cosine_similarity che è agnostica alla dimensione.
    print(f"Avvio calcolo Baseline su {Config.num_episodes} episodi...")
    
    results = evaluate_few_shot(
        model=model, 
        fewshot_iterator=iterator, 
        transform=transform, 
        device=device, 
        num_episodes=Config.num_episodes
    )

    # 5. Stampa dei Risultati
    print("\n=== Risultati Baseline (ResNet50 'Off-the-shelf') ===")
    for metrica, valore in results.items():
        print(f"{metrica.capitalize():<15}: {valore:.4f}")

if __name__ == "__main__":
    main()